### Third task
Extract product characteristics from the specified files and generate an Excel file with product serial numbers and their characteristics.

In [1]:
import os
from collections import defaultdict
def get_files_by_format(folder_path):
    files_by_format = defaultdict(list)

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            
            _, extension = os.path.splitext(file)
            format_name = extension.lower() if extension else '<empty>'
            
            files_by_format[format_name].append(file_path)
    
    return dict(files_by_format)

In [2]:
files_by_format = get_files_by_format("./files")

In [7]:
from docx import Document
def docx_to_markdown_tables_list(file_path):
    doc = Document(file_path)
    
    data = []
    for table in doc.tables:
        for row in table.rows:
            row_data = [cell.text.strip() for cell in row.cells]
            data.append(row_data)

            
    headers = data[0]
    table_lines = [
        '| ' + ' | '.join(headers) + ' |',
        '| ' + ' | '.join(['---'] * len(headers)) + ' |'
    ]
    
    for row in data[1:]:
        table_lines.append('| ' + ' | '.join(row) + ' |')
    
    return '\n'.join(table_lines)

In [13]:
import pandas as pd

lamp = pd.read_excel("./tasks/Памятка.xlsx", sheet_name="Светильники")
battery = pd.read_excel("./tasks/Памятка.xlsx", sheet_name="Аккумуляторы")

In [23]:
lamp_characteristics = lamp.columns.to_list()
battery_characteristics = battery.columns.to_list()

product_characteristics = {"Lamp": lamp_characteristics, "Battery": battery_characteristics}

In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

def extract_columns_info_from_text(text, json_schema: dict):
    llm = ChatOpenAI(model="gpt-4o-mini")
    prompt = PromptTemplate(template="""[TASK] Extract structured information from the markdown table in text below. \
If there are many objects, then return a list of JSON objects.

[TEXT]
{text}

[OUTPUT REQUIREMENTS]
- Format: list of JSONs
- Schema:
{json_schema}

[INSTRUCTIONS]
1. Strictly adhere to the specified schema
2. Use the correct data types and metrics
3. Use null for missing data
4. Do not add explanations to the output
5. Check the validity of the JSON

[OUTPUT]""", input_variables=["text", "json_schema"])
    
    chain = prompt | llm | JsonOutputParser()

    response = chain.invoke({"json_schema": json_schema, "text": text})
    
    return response

In [35]:
data_for_df = {"Lamp": [], "Battery": []}

In [38]:
for product, characteristics in product_characteristics.items():
    json_schema = {characteristic : f"data{i}" for i, characteristic in enumerate(characteristics)}
    for file_path in files_by_format[".docx"]:
        markdown_text = docx_to_markdown_tables_list(file_path)
        result = extract_columns_info_from_text(markdown_text, json_schema)
        for i, characteristics_from_llm in enumerate(result):
            data_for_df[product].append([i+1] + [val for key, val in characteristics_from_llm.items()] + [file_path])


In [37]:
data_for_df

{'Lamp': [], 'Battery': []}

For pdf using llm

In [39]:
import pymupdf4llm
def convert_pdf_to_md_text(file_path):
    md_text = pymupdf4llm.to_markdown(file_path)
    return md_text

In [40]:
for product, characteristics in product_characteristics.items():
    json_schema = {characteristic : f"data{i}" for i, characteristic in enumerate(characteristics)}
    md_text = convert_pdf_to_md_text(files_by_format[".pdf"][0])
    result = extract_columns_info_from_text(md_text, json_schema)
    for i, characteristics_from_llm in enumerate(result):
        data_for_df[product].append([i+1] + [val for key, val in characteristics_from_llm.items()] + [file_path])

In [41]:
lamp_df = pd.DataFrame(data_for_df["Lamp"], columns=["№"] + [val for val in product_characteristics["Lamp"]] + ["document_from"])
battery_df = pd.DataFrame(data_for_df["Battery"], columns=["№"] + [val for val in product_characteristics["Battery"]] + ["document_from"])

In [42]:
lamp_df.to_excel("./results/extract3_lamp.xlsx", index=False)
battery_df.to_excel("./results/extract3_battery.xlsx", index=False)